### Paquetes necesarios

In [1]:
from algorithm.object_detector import YOLOv7
from utils.detections import draw
import json
import cv2
import numpy as np
from tqdm import tqdm

### Funciones auxiliares

##### Método para el filtrado de las detecciones y el cálculo del punto medio del objeto detectado. 

In [2]:
def filter_detections(detections, confidence):
    filtered_detections = []
    for object_detected in detections:
        if object_detected['confidence'] > confidence:
            object_detected['cx'] = object_detected['x'] + int(object_detected['width'] / 2)
            object_detected['cy'] = object_detected['y'] + int(object_detected['height'] / 2)
            filtered_detections.append(object_detected)
    return filtered_detections

##### Método para comprobar la existencia del id de un objeto dentro de una lista

In [3]:
def id_exist(list_ids, id):
    for item in list_ids:
        if id == item:
            return True
    return False

### Código principal

In [5]:
yolov7 = YOLOv7()

#Cambien 'yolov7-e6e.pt' por 'coco.weights' para probar el modelo de easy-yolov7
yolov7.load('yolov7-e6e.pt', classes='coco.yaml', device='cpu') # use 'gpu' for CUDA GPU inference
confidence = 0.70

video = cv2.VideoCapture('traffic-video-1080.mp4')

width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
frames_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
output = cv2.VideoWriter('output-640-easy.mp4', fourcc, fps, (width, height))

region_1 = [(387, 441), (758, 441), (698, 789), (159, 736)]
region_2 = [(828, 385), (986, 377), (1069, 739), (830, 738)]
region_3 = [(1618, 545), (1845, 620), (1699, 802), (1397, 704)]
region_4 = [(206, 207), (417, 221), (350, 366), (96, 339)]

if video.isOpened() == False:
	print('[!] error opening the video')

pbar = tqdm(total=frames_count, unit=' frames', dynamic_ncols=True, position=0, leave=True)

list_id_cars = []
list_id_trucks = []
list_id_bus = []
list_id_motorcycle = []
list_id_persons = []

try:
    while video.isOpened():

        ret, frame = video.read()

        if not ret:
          break

        detections = yolov7.detect(frame, track=True)
        detections = filter_detections(detections, confidence)
        detected_frame = draw(frame, detections)

        cv2.polylines(detected_frame, [np.array(region_1)], True, (255, 255, 255), 4)
        cv2.polylines(detected_frame, [np.array(region_2)], True, (255, 255, 255), 4)
        cv2.polylines(detected_frame, [np.array(region_3)], True, (255, 255, 255), 4)
        cv2.polylines(detected_frame, [np.array(region_4)], True, (0, 0, 0), 4)
 
        for box in detections:

            cv2.circle(detected_frame, (box['cx'], box['cy']), 5, (0, 0, 255), -1)
        
            if box['class'] == "car":
                inside1 = cv2.pointPolygonTest(np.array(region_1), (box['cx'], box['cy']), False)
                inside2 = cv2.pointPolygonTest(np.array(region_2), (box['cx'], box['cy']), False)
                inside3 = cv2.pointPolygonTest(np.array(region_3), (box['cx'], box['cy']), False)

                if (inside1 > 0) or (inside2 > 0) or (inside3 > 0):
                    if (id_exist(list_id_cars, box['id']) == False):
                        list_id_cars.append(box['id'])

            elif box['class'] == "truck":
                inside1 = cv2.pointPolygonTest(np.array(region_1), (box['cx'], box['cy']), False)
                inside2 = cv2.pointPolygonTest(np.array(region_2), (box['cx'], box['cy']), False)
                inside3 = cv2.pointPolygonTest(np.array(region_3), (box['cx'], box['cy']), False)

                if (inside1 > 0) or (inside2 > 0) or (inside3 > 0):
                    if (id_exist(list_id_trucks, box['id']) == False):
                        list_id_trucks.append(box['id'])

            elif box['class'] == "bus":
                inside1 = cv2.pointPolygonTest(np.array(region_1), (box['cx'], box['cy']), False)
                inside2 = cv2.pointPolygonTest(np.array(region_2), (box['cx'], box['cy']), False)
                inside3 = cv2.pointPolygonTest(np.array(region_3), (box['cx'], box['cy']), False)

                if (inside1 > 0) or (inside2 > 0) or (inside3 > 0):
                    if (id_exist(list_id_bus, box['id']) == False):
                        list_id_bus.append(box['id'])

            elif box['class'] == "motorcycle":
                inside1 = cv2.pointPolygonTest(np.array(region_1), (box['cx'], box['cy']), False)
                inside2 = cv2.pointPolygonTest(np.array(region_2), (box['cx'], box['cy']), False)
                inside3 = cv2.pointPolygonTest(np.array(region_3), (box['cx'], box['cy']), False)

                if (inside1 > 0) or (inside2 > 0) or (inside3 > 0):
                    if (id_exist(list_id_motorcycle, box['id']) == False):
                        list_id_motorcycle.append(box['id'])

            elif box['class'] == "person":
                inside4 = cv2.pointPolygonTest(np.array(region_4), (box['cx'], box['cy']), False)

                if inside4 > 0:
                    if (id_exist(list_id_persons, box['id']) == False):
                        list_id_persons.append(box['id'])

        car_count = len(list_id_cars)
        truck_count = len(list_id_trucks)
        bus_count = len(list_id_bus)
        motorcycle_count = len(list_id_motorcycle)
        person_count = len(list_id_persons)


        cv2.rectangle(detected_frame, (1443,32), (1852, 242), (0, 0, 0), -1)
        cv2.putText(detected_frame, "Cars: " + str(car_count), (1461, 64), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)
        cv2.putText(detected_frame, "Trucks: " + str(truck_count), (1461, 104), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)
        cv2.putText(detected_frame, "Bus: " + str(bus_count), (1461, 144), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)
        cv2.putText(detected_frame, "Motorcycles: " + str(motorcycle_count), (1461, 184), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)
        cv2.putText(detected_frame, "Persons: " + str(person_count), (1461, 224), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)

        output.write(detected_frame)
        pbar.update(1)
        
except KeyboardInterrupt:
    pass
    
pbar.close()
video.release()
print('[+] video closed')
output.release()
yolov7.unload()

  0%|          | 33/9186 [00:10<48:58,  3.12 frames/s] 

[+] video closed
